In [1]:
import numpy as np
import json
import pandas as pd
import chardet


### Kaggle DataSet

In [35]:
df = pd.read_csv('Bitext_Sample_Customer_Service_Training_Dataset.csv', encoding='utf-8')
df = df.drop(columns=['flags'])
df.head()

,utterance,category,intent
0,I have problems with canceling an order,ORDER,cancel_order
1,how can I find information about canceling ord...,ORDER,cancel_order
2,I need help with canceling the last order,ORDER,cancel_order
3,could you help me cancelling the last order I ...,ORDER,cancel_order
4,problem with cancelling an order I made,ORDER,cancel_order


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8175 entries, 0 to 8174
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   utterance  8175 non-null   object
 1   category   8175 non-null   object
 2   intent     8175 non-null   object
dtypes: object(3)
memory usage: 191.7+ KB


In [37]:
order_df = df[df['category'] == 'ORDER']
order_df.head()

,utterance,category,intent
0,I have problems with canceling an order,ORDER,cancel_order
1,how can I find information about canceling ord...,ORDER,cancel_order
2,I need help with canceling the last order,ORDER,cancel_order
3,could you help me cancelling the last order I ...,ORDER,cancel_order
4,problem with cancelling an order I made,ORDER,cancel_order


In [38]:
print(df['intent'].nunique())

27


In [30]:
print(order_df['intent'].nunique())

4


In [111]:
# Création d'une DataFrame vide
new_df = pd.DataFrame(columns=['patterns', 'tag', 'responses']) 

In [112]:
new_df.head()

,patterns,tag,responses


In [113]:

intent = []

for i in range(len(df)):
    current_intent = df.intent.iloc[i]
    
    if current_intent not in intent:
        intent.append(current_intent)
        new_df.loc[len(new_df)] = [[df.utterance.iloc[i]], df.category.iloc[i], current_intent]
    else:
        index = new_df.index[new_df.responses == current_intent].tolist()
        if index:
            new_df.at[index[0], 'patterns'].append(df.utterance.iloc[i])


In [114]:
new_df.head()

,patterns,tag,responses
0,"[I have problems with canceling an order, how ...",ORDER,cancel_order
1,[can you help me to add something to an order?...,ORDER,change_order
2,[I have an error correcting the shipping addre...,SHIPPING_ADDRESS,change_shipping_address
3,[I don't know what I need to do to check the e...,CANCELLATION_FEE,check_cancellation_fee
4,[I do not know how I can find the invoice from...,INVOICE,check_invoice


In [115]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   patterns   27 non-null     object
 1   tag        27 non-null     object
 2   responses  27 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


In [116]:
print(new_df['tag'].nunique())

11


In [93]:
new_df = new_df.sort_values(by='tag')
new_df = new_df.reset_index(drop=True)
new_df.head()

,patterns,tag,response
0,"[help me changing to a new account, can ya hel...",ACCOUNT,switch_account
1,[I do not know how I could edit the informatio...,ACCOUNT,edit_account
2,[what do I have to do to report a registration...,ACCOUNT,registration_problems
3,"[tell me about a forgotten password, where can...",ACCOUNT,recover_password
4,"[I want assistance deleting my account, how ca...",ACCOUNT,delete_account


In [94]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   patterns  27 non-null     object
 1   tag       27 non-null     object
 2   response  27 non-null     object
dtypes: object(3)
memory usage: 780.0+ bytes


In [126]:
rep = new_df.responses.unique()
rep

array(['cancel_order', 'change_order', 'change_shipping_address',
       'check_cancellation_fee', 'check_invoice', 'check_payment_methods',
       'check_refund_policy', 'complaint', 'contact_customer_service',
       'contact_human_agent', 'create_account', 'delete_account',
       'delivery_options', 'delivery_period', 'edit_account',
       'get_invoice', 'get_refund', 'newsletter_subscription',
       'payment_issue', 'place_order', 'recover_password',
       'registration_problems', 'review', 'set_up_shipping_address',
       'switch_account', 'track_order', 'track_refund'], dtype=object)

In [128]:
liste = []

for i in range(len(new_df)):
    a,b = new_df.tag.iloc[i], new_df.responses.iloc[i]
    if b not in liste:
        liste.append([a,b])

In [130]:
print(liste)
print(len(liste))

[['ORDER', 'cancel_order'], ['ORDER', 'change_order'], ['SHIPPING_ADDRESS', 'change_shipping_address'], ['CANCELLATION_FEE', 'check_cancellation_fee'], ['INVOICE', 'check_invoice'], ['PAYMENT', 'check_payment_methods'], ['REFUND', 'check_refund_policy'], ['FEEDBACK', 'complaint'], ['CONTACT', 'contact_customer_service'], ['CONTACT', 'contact_human_agent'], ['ACCOUNT', 'create_account'], ['ACCOUNT', 'delete_account'], ['DELIVERY', 'delivery_options'], ['DELIVERY', 'delivery_period'], ['ACCOUNT', 'edit_account'], ['INVOICE', 'get_invoice'], ['REFUND', 'get_refund'], ['NEWSLETTER', 'newsletter_subscription'], ['PAYMENT', 'payment_issue'], ['ORDER', 'place_order'], ['ACCOUNT', 'recover_password'], ['ACCOUNT', 'registration_problems'], ['FEEDBACK', 'review'], ['SHIPPING_ADDRESS', 'set_up_shipping_address'], ['ACCOUNT', 'switch_account'], ['ORDER', 'track_order'], ['REFUND', 'track_refund']]
27


In [ ]:

couple = []
for i in range(len(new_df)):
    tag,rep = new_df.tag.iloc[i], new_df.responses.iloc[i]
    if [tag, rep] not in couple:
        print(f'{tag} : {rep}')
        couple.append([tag, rep])a




ORDER : cancel_order
ORDER : change_order
SHIPPING_ADDRESS : change_shipping_address
CANCELLATION_FEE : check_cancellation_fee
INVOICE : check_invoice
PAYMENT : check_payment_methods
REFUND : check_refund_policy
FEEDBACK : complaint
CONTACT : contact_customer_service
CONTACT : contact_human_agent
ACCOUNT : create_account
ACCOUNT : delete_account
DELIVERY : delivery_options
DELIVERY : delivery_period
ACCOUNT : edit_account
INVOICE : get_invoice
REFUND : get_refund
NEWSLETTER : newsletter_subscription
PAYMENT : payment_issue
ORDER : place_order
ACCOUNT : recover_password
ACCOUNT : registration_problems
FEEDBACK : review
SHIPPING_ADDRESS : set_up_shipping_address
ACCOUNT : switch_account
ORDER : track_order
REFUND : track_refund


In [ ]:
def update_unique_tags(df):
    """
    Met à jour la colonne 'tag' de df afin que chaque couple (tag, responses)
    soit unique en ajoutant un suffixe numérique. La fonction élimine aussi
    les doublons exacts.
    """
    # 1. Éliminer les doublons exacts sur les colonnes 'tag' et 'responses'
    df_unique = df.drop_duplicates(subset=['tag', 'responses']).copy()
    
    # 2. Remplacer chaque tag par tag_suffix (ex : ORDER_0, ORDER_1, etc.)
    tag_counts = {}  # Dictionnaire pour compter les occurrences de chaque tag d'origine
    new_tags = []    # Liste pour stocker les nouveaux tags
    
    for idx, row in df_unique.iterrows():
        original_tag = row['tag']
        count = tag_counts.get(original_tag, 0)
        # Construit le nouveau tag avec suffixe
        new_tag = f"{original_tag}_{count}"
        tag_counts[original_tag] = count + 1
        new_tags.append(new_tag)
    
    df_unique['tag'] = new_tags
    return df_unique

new_df = update_unique_tags(new_df)


                                             patterns                 tag  \
0   [I have problems with canceling an order, how ...             ORDER_0   
1   [can you help me to add something to an order?...             ORDER_1   
2   [I have an error correcting the shipping addre...  SHIPPING_ADDRESS_0   
3   [I don't know what I need to do to check the e...  CANCELLATION_FEE_0   
4   [I do not know how I can find the invoice from...           INVOICE_0   
5   [I need assistance seeing your available payme...           PAYMENT_0   
6   [checking your refund policy, I don't know how...            REFUND_0   
7   [I need to file a complaint, I want assistance...          FEEDBACK_0   
8   [I have to check at what time I can call custo...           CONTACT_0   
9   [help to contact an agent, speak to a goddamn ...           CONTACT_1   
10  [how do i create a goddamn new account, i do n...           ACCOUNT_0   
11  [I want assistance deleting my account, how ca...           ACCOUNT_1   

In [137]:

new_df.head()

,patterns,tag,responses
0,"[I have problems with canceling an order, how ...",ORDER_0,cancel_order
1,[can you help me to add something to an order?...,ORDER_1,change_order
2,[I have an error correcting the shipping addre...,SHIPPING_ADDRESS_0,change_shipping_address
3,[I don't know what I need to do to check the e...,CANCELLATION_FEE_0,check_cancellation_fee
4,[I do not know how I can find the invoice from...,INVOICE_0,check_invoice


In [138]:
print(new_df['tag'].nunique())

27


In [139]:
# Convertir la DataFrame en liste de dictionnaires
intents_list = new_df.to_dict(orient="records")

# Créer le dictionnaire final avec la clé "intents"
data = {"intents": intents_list}

# Enregistrer dans un fichier JSON avec une indentation pour une meilleure lisibilité
with open("intents2.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

print("Le fichier JSON a été enregistré avec succès.")

Le fichier JSON a été enregistré avec succès.


In [118]:
%run v4.py

Epoch 1, loss: 3.8539
Epoch 2, loss: 3.5732
Epoch 3, loss: 3.0902
Epoch 4, loss: 2.5207
Epoch 5, loss: 1.9892
Epoch 6, loss: 1.6496
Epoch 7, loss: 1.4498
Epoch 8, loss: 1.2288
Epoch 9, loss: 1.0142
Epoch 10, loss: 0.9487
Epoch 11, loss: 0.8306
Epoch 12, loss: 0.7672
Epoch 13, loss: 0.6731
Epoch 14, loss: 0.6255
Epoch 15, loss: 0.6257
Epoch 16, loss: 0.5798
Epoch 17, loss: 0.4763
Epoch 18, loss: 0.4631
Epoch 19, loss: 0.4823
Epoch 20, loss: 0.4624
Epoch 21, loss: 0.4327
Epoch 22, loss: 0.4510
Epoch 23, loss: 0.3829
Epoch 24, loss: 0.3493
Epoch 25, loss: 0.3082
Epoch 26, loss: 0.2984
Epoch 27, loss: 0.3400
Epoch 28, loss: 0.3007
Epoch 29, loss: 0.3345
Epoch 30, loss: 0.2880
Epoch 31, loss: 0.3056
Epoch 32, loss: 0.2652
Epoch 33, loss: 0.2582
Epoch 34, loss: 0.2768
Epoch 35, loss: 0.2639
Epoch 36, loss: 0.2690
Epoch 37, loss: 0.2220
Epoch 38, loss: 0.1954
Epoch 39, loss: 0.2243
Epoch 40, loss: 0.2475
Epoch 41, loss: 0.2273
Epoch 42, loss: 0.2174
Epoch 43, loss: 0.1960
Epoch 44, loss: 0.19

In [ ]:
%run v2.py

C:\Users\delah\Documents\GitHub\AI-Chatbot\v2.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path))


User: hey
Bot: Hello!
User: how are you buddy ?
Bot: Yes, I'm an AI chatbot, how can I assist you?
User: heya
Bot: Hey! How can I assist you today?
User: i fells bad about lions
Bot: Famous lions include Simba from 'The Lion King' and Cecil the lion.


In [122]:
%run v2.py

C:\Users\delah\Documents\GitHub\AI-Chatbot\v2.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path))


User: 
Bot: Hey! How can I assist you today?
User: sqlut
Bot: They primarily inhabit savannahs and grasslands.
User: salut ca va
Bot: Hi there!
User: do you watch nhl ?
Bot: Popular NHL teams include the Toronto Maple Leafs, Boston Bruins, and Montreal Canadiens.


In [146]:
%run v4.py

Epoch 1, loss: 1.1991
Epoch 2, loss: 0.3320
Epoch 3, loss: 0.1959
Epoch 4, loss: 0.1308
Epoch 5, loss: 0.1107
Epoch 6, loss: 0.0872
Epoch 7, loss: 0.0772
Epoch 8, loss: 0.0641
Epoch 9, loss: 0.0699
Epoch 10, loss: 0.0598
Epoch 11, loss: 0.0584
Epoch 12, loss: 0.0449
Epoch 13, loss: 0.0482
Epoch 14, loss: 0.0412
Epoch 15, loss: 0.0418
Epoch 16, loss: 0.0418
Epoch 17, loss: 0.0422
Epoch 18, loss: 0.0419
Epoch 19, loss: 0.0361
Epoch 20, loss: 0.0421
Epoch 21, loss: 0.0343
Epoch 22, loss: 0.0302
Epoch 23, loss: 0.0386
Epoch 24, loss: 0.0309
Epoch 25, loss: 0.0424
Epoch 26, loss: 0.0379
Epoch 27, loss: 0.0361
Epoch 28, loss: 0.0344
Epoch 29, loss: 0.0338
Epoch 30, loss: 0.0283
Epoch 31, loss: 0.0325
Epoch 32, loss: 0.0352
Epoch 33, loss: 0.0321
Epoch 34, loss: 0.0277
Epoch 35, loss: 0.0341
Epoch 36, loss: 0.0337
Epoch 37, loss: 0.0310
Epoch 38, loss: 0.0295
Epoch 39, loss: 0.0323
Epoch 40, loss: 0.0330
Epoch 41, loss: 0.0369
Epoch 42, loss: 0.0290
Epoch 43, loss: 0.0320
Epoch 44, loss: 0.02